In [1]:
import spacy

In [2]:
spacy.__version__

'3.0.3'

In [3]:
nlp = spacy.load("en_core_web_trf")

In [4]:
nlp.pipe_names

['transformer', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']

In [5]:
nlp.meta['name']

'core_web_trf'

In [6]:
#nlp("hello there")._.trf_data

In [7]:
from pathlib import Path

import os
import sys

current_path = Path(os.getcwd())
sys.path.append(str(current_path.parent))

In [8]:
import vectorian

from vectorian.importers import NovelImporter
from vectorian.session import LabSession
from vectorian.embeddings import SpacyTransformerEmbedding

my_embedding = SpacyTransformerEmbedding(nlp)

# contextual embeddings
im = NovelImporter(nlp, embeddings=[my_embedding])

#im = NovelImporter(nlp)
doc = im("/Users/arbeit/Wise Children Mini.txt")

Matching checksum for /Users/arbeit/Projects/vectorian-2021/vectorian/core/cpp/core.cpp --> not compiling


Importing /Users/arbeit/Wise Children Mini.txt: 100%|██████████| 1/1 [00:00<00:00,  8.07it/s]


In [9]:
doc.save("/Users/arbeit/Desktop/my_example_doc")

In [10]:
from vectorian.corpus.document import Document
doc = Document.load("/Users/arbeit/Desktop/my_example_doc")

In [11]:
# note: you may also specify StackedEmbedding here.
session = LabSession(
    [doc],
    embeddings=[my_embedding],
    token_mappings="default")

In [12]:
from vectorian.importers import StringImporter
from vectorian.session import LabSession
from vectorian.metrics import CosineSimilarity, TokenSimilarity
from vectorian.metrics import AlignmentSentenceSimilarity
from vectorian.alignment import WatermanSmithBeyer, ConstantGapCost

metric = AlignmentSentenceSimilarity(
    token_metric=TokenSimilarity(my_embedding, CosineSimilarity()),
    alignment=WatermanSmithBeyer(gap=ConstantGapCost(0), zero=0.1))

index = session.partition("sentence").index(metric, nlp)

In [13]:
def debug(hook, args):
    print(hook)
    if hook == 'contextual_similarity_matrix':
        print(args)
    if hook == 'alignment/waterman-smith-beyer':
        print(args)

index.find('you', n=1)  #, debug=debug)

FloatProgress(value=0.0, layout=Layout(width='100%'), max=1.0)

AttributeError: 'Dataset' object has no attribute 'value'

In [12]:
import json
with open("/Users/arbeit/Desktop/debug.json", "r") as f:
    data = json.loads(f.read())

In [27]:
np.array(data["a"])[0][:10]

array([ 1.11921835, -1.50500298, -0.47002545,  0.72598863,  1.25613248,
       -1.59027803, -0.11668806, -0.79666078,  0.65255964, -0.2121384 ])

In [26]:
np.array(data["b"])[-2][:10]

array([ 0.49695939, -1.57025933,  0.48457262, -0.32552609, -1.09349322,
       -1.80017662,  0.84969461, -0.1012421 ,  0.49484468, -0.32485196])

In [29]:
import numpy as np
import json
with open("/Users/arbeit/Desktop/debug2.json", "r") as f:
    data = json.loads(f.read())
np.array(data).shape

(2, 10)